In [ ]:
import cv2
import numpy as np
import argparse

In [ ]:
parser = argparse.ArgumentParser(description="Generate Surrounding Camera Bird Eye View")
parser.add_argument('-fw', '--FRAME_WIDTH', default=1280, type=int, help='Camera Frame Width')
parser.add_argument('-fh', '--FRAME_HEIGHT', default=1024, type=int, help='Camera Frame Height')
parser.add_argument('-bw', '--BEV_WIDTH', default=1000, type=int, help='BEV Frame Width')
parser.add_argument('-bh', '--BEV_HEIGHT', default=1000, type=int, help='BEV Frame Height')
parser.add_argument('-cw', '--CAR_WIDTH', default=250, type=int, help='Car Frame Width')
parser.add_argument('-ch', '--CAR_HEIGHT', default=400, type=int, help='Car Frame Height')
parser.add_argument('-fs', '--FOCAL_SCALE', default=1, type=float, help='Camera Undistort Focal Scale')
parser.add_argument('-ss', '--SIZE_SCALE', default=2, type=float, help='Camera Undistort Size Scale')
parser.add_argument('-blend','--BLEND_FLAG', default=False, type=bool, help='Blend BEV Image (Ture/False)')
parser.add_argument('-balance','--BALANCE_FLAG', default=False, type=bool, help='Balance BEV Image (Ture/False)')
args = parser.parse_args([])

In [ ]:
# args.FRAME_WIDTH = 1280     # 카메라 원본 이미지 너비
# args.FRAME_HEIGHT = 1024    # 카메라 원본 이미지 높이
# args.BEV_WIDTH = 1000       # 최종 조감도 모자이크 폭
# args.BEV_HEIGHT = 1000      # 최종 조감도 모자이크 높이
# args.CAR_WIDTH = 250        # 조감도 중앙의 차량 폭 (자동차 사진에 해당)
# args.CAR_HEIGHT = 400       # 조감도 중앙의 차량 높이(차량 사진에 해당)
# args.FOCAL_SCALE = 1        # 왜곡 제거 시 카메라 초점 거리 스케일링 계수(내부 및 외부 교정 타이밍과 일치)
# args.SIZE_SCALE = 2         # 왜곡 제거 시 카메라 크기 배율 인수(내부 및 외부 보정 타이밍과 일치)
# args.BLEND_FLAG = True      # 조감도 스티칭에 이미지 융합을 사용할지 여부
# args.BALANCE_FLAG = True    # 조감도 스티칭에 이미지 균형을 사용할지 여부

In [ ]:
# 직접 할당을 사용하면 args를 자주 읽지 않아도 됩니다.
FRAME_WIDTH = args.FRAME_WIDTH
FRAME_HEIGHT = args.FRAME_HEIGHT
BEV_WIDTH = args.BEV_WIDTH
BEV_HEIGHT = args.BEV_HEIGHT
CAR_WIDTH = args.CAR_WIDTH
CAR_HEIGHT = args.CAR_HEIGHT
FOCAL_SCALE = args.FOCAL_SCALE
SIZE_SCALE = args.SIZE_SCALE

In [ ]:
# 이미지 보충 검정색 테두리
def padding(img,width,height):
    H = img.shape[0]
    W = img.shape[1]
    top = (height - H) // 2 
    bottom = (height - H) // 2 
    if top + bottom + H < height:
        bottom += 1
    left = (width - W) // 2 
    right = (width - W) // 2 
    if left + right + W < width:
        right += 1
    img = cv2.copyMakeBorder(img, top, bottom, left, right,
                             cv2.BORDER_CONSTANT, value = (0,0,0)) 
    return img

In [ ]:
# 컬러 밸런스(화이트 밸런스)
def color_balance(image):
    b, g, r = cv2.split(image)
    B = np.mean(b)
    G = np.mean(g)
    R = np.mean(r)
    K = (R + G + B) / 3
    Kb = K / B
    Kg = K / G
    Kr = K / R
    cv2.addWeighted(b, Kb, 0, 0, 0, b)
    cv2.addWeighted(g, Kg, 0, 0, 0, g)
    cv2.addWeighted(r, Kr, 0, 0, 0, r)
    return cv2.merge([b,g,r])

In [ ]:
# 밝기 균형: 4개의 입력 영상을 기준으로 평균 밝기를 계산하여 HSV 채널로 변환합니다.
def luminance_balance(images):
    [front,back,left,right] = [cv2.cvtColor(image,cv2.COLOR_BGR2HSV) 
                               for image in images]
    hf, sf, vf = cv2.split(front)
    hb, sb, vb = cv2.split(back)
    hl, sl, vl = cv2.split(left)
    hr, sr, vr = cv2.split(right)
    V_f = np.mean(vf)
    V_b = np.mean(vb)
    V_l = np.mean(vl)
    V_r = np.mean(vr)
    V_mean = (V_f + V_b + V_l +V_r) / 4
    vf = cv2.add(vf,(V_mean - V_f))
    vb = cv2.add(vb,(V_mean - V_b))
    vl = cv2.add(vl,(V_mean - V_l))
    vr = cv2.add(vr,(V_mean - V_r))
    front = cv2.merge([hf,sf,vf])
    back = cv2.merge([hb,sb,vb])
    left = cv2.merge([hl,sl,vl])
    right = cv2.merge([hr,sr,vr])
    images = [front,back,left,right]
    images = [cv2.cvtColor(image,cv2.COLOR_HSV2BGR) for image in images]
    return images

In [ ]:
class Camera:                   # 카메라 클래스: 매개변수 읽기, 왜곡 제거 및 호모그래피 변환
    def __init__(self, name):
        # 내부 매개변수, 왜곡 벡터 및 외부 매개변수의 npy 파일 읽기
        self.camera_mat = np.load('./data/{}/camera_{}_K.npy'.format(name,name))
        self.dist_coeff = np.load('./data/{}/camera_{}_D.npy'.format(name,name))
        self.homography = np.load('./data/{}/camera_{}_H.npy'.format(name,name))
        self.camera_mat_dst = self.get_camera_mat_dst()
        self.undistort_maps = self.get_undistort_maps()
        self.bev_maps = self.get_bev_maps()
    
    # 왜곡 제거를 위한 새로운 내부 카메라 매개변수 매트릭스를 얻습니다. 초점 거리와 프레임을 수정할 수 있습니다.
    def get_camera_mat_dst(self):
        camera_mat_dst = self.camera_mat.copy()
        camera_mat_dst[0][0] *= FOCAL_SCALE
        camera_mat_dst[1][1] *= FOCAL_SCALE
        camera_mat_dst[0][2] = FRAME_WIDTH / 2 * SIZE_SCALE
        camera_mat_dst[1][2] = FRAME_HEIGHT / 2 * SIZE_SCALE
        return camera_mat_dst
    
    # 왜곡 제거 매핑 행렬 가져오기
    def get_undistort_maps(self):
        undistort_maps = cv2.fisheye.initUndistortRectifyMap(
                    self.camera_mat, self.dist_coeff, 
                    np.eye(3, 3), self.camera_mat_dst,
                    (int(FRAME_WIDTH * SIZE_SCALE), int(FRAME_HEIGHT * SIZE_SCALE)), cv2.CV_16SC2)
        return undistort_maps
    
    # 편향 매핑 행렬에 대해 호모그래피 변환 수행
    def get_bev_maps(self):
        map1 = self.warp_homography(self.undistort_maps[0])
        map2 = self.warp_homography(self.undistort_maps[1])
        return (map1, map2)
    
    # 원본 이미지 왜곡
    def undistort(self, img):
        return cv2.remap(img, *self.undistort_maps, interpolation = cv2.INTER_LINEAR)
        
    # 이미지에서 직접 호모그래피 변환 수행
    def warp_homography(self, img):
        return cv2.warpPerspective(img, self.homography, (BEV_WIDTH,BEV_HEIGHT))
    
    # 호모그래피 변환 후 매핑 행렬을 사용하여 더 빠른 리맵 변환을 수행합니다.
    def raw2bev(self, img):
        return cv2.remap(img, *self.bev_maps, interpolation = cv2.INTER_LINEAR)

In [ ]:
class Mask:                     # 4개 이미지의 마스크를 직접 연결합니다.
    def __init__(self, name):
        self.mask = self.get_mask(name)
    
    # 앞, 뒤, 왼쪽, 오른쪽 4개 마스크의 미리 설정된 좌표점
    def get_points(self, name):
        if name == 'front':
            points = np.array([
                [0, 0],
                [BEV_WIDTH, 0], 
                [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2],
                [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2]
            ]).astype(np.int32)
        elif name == 'back':
            points = np.array([
                [0, BEV_HEIGHT],
                [BEV_WIDTH, BEV_HEIGHT],
                [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2], 
                [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2]
            ]).astype(np.int32)
        elif name == 'left':
            points = np.array([
                [0, 0],
                [0, BEV_HEIGHT], 
                [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2],
                [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2]
            ]).astype(np.int32)
        elif name == 'right':
            points = np.array([
                [BEV_WIDTH, 0],
                [BEV_WIDTH, BEV_HEIGHT], 
                [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2], 
                [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2]
            ]).astype(np.int32)
        else:
            raise Exception("name should be front/back/left/right")
        return points
    
    # 마스크를 하나하나 채워주세요
    def get_mask(self, name):
        mask = np.zeros((BEV_HEIGHT,BEV_WIDTH), dtype=np.uint8)
        points = self.get_points(name)
        return cv2.fillPoly(mask, [points], 255)
    
    # 마스크 이후의 이미지를 얻기 위한 비트별 AND 계산
    def __call__(self, img):
        return cv2.bitwise_and(img, img, mask=self.mask)

In [ ]:
class BlendMask:                # 4개 이미지의 마스크 융합 및 접합
    def __init__(self,name):
        mf = self.get_mask('front')
        mb = self.get_mask('back')
        ml = self.get_mask('left')
        mr = self.get_mask('right')
        self.get_lines()
        if name == 'front':
            mf = self.get_blend_mask(mf, ml, self.lineFL, self.lineLF)
            mf = self.get_blend_mask(mf, mr, self.lineFR, self.lineRF)
            self.mask = mf
        if name == 'back':
            mb = self.get_blend_mask(mb, ml, self.lineBL, self.lineLB)
            mb = self.get_blend_mask(mb, mr, self.lineBR, self.lineRB)
            self.mask = mb
        if name == 'left':
            ml = self.get_blend_mask(ml, mf, self.lineLF, self.lineFL)
            ml = self.get_blend_mask(ml, mb, self.lineLB, self.lineBL)
            self.mask = ml
        if name == 'right':
            mr = self.get_blend_mask(mr, mf, self.lineRF, self.lineFR)
            mr = self.get_blend_mask(mr, mb, self.lineRB, self.lineBR)
            self.mask = mr
        self.weight = np.repeat(self.mask[:, :, np.newaxis], 3, axis=2) / 255.0
        self.weight = self.weight.astype(np.float32)
    
    # 앞, 뒤, 왼쪽, 오른쪽 4개의 마스크의 미리 설정된 좌표점은 직접 접합된 마스크에 비해 겹치는 부분이 있으며, 값을 수정하면 겹치는 범위를 변경할 수 있습니다.
    def get_points(self, name):
        if name == 'front':
            points = np.array([
                [0, 0],
                [BEV_WIDTH, 0], 
                [BEV_WIDTH, BEV_HEIGHT/5], 
                [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2],
                [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2],
                [0, BEV_HEIGHT/5], 
            ]).astype(np.int32)
        elif name == 'back':
            points = np.array([
                [0, BEV_HEIGHT],
                [BEV_WIDTH, BEV_HEIGHT],
                [BEV_WIDTH, BEV_HEIGHT - BEV_HEIGHT/5],
                [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2], 
                [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2],
                [0, BEV_HEIGHT - BEV_HEIGHT/5],
            ]).astype(np.int32)
        elif name == 'left':
            points = np.array([
                [0, 0],
                [0, BEV_HEIGHT], 
                [BEV_WIDTH/5, BEV_HEIGHT], 
                [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2],
                [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2],
                [BEV_WIDTH/5, 0]
            ]).astype(np.int32)
        elif name == 'right':
            points = np.array([
                [BEV_WIDTH, 0],
                [BEV_WIDTH, BEV_HEIGHT], 
                [BEV_WIDTH - BEV_WIDTH/5, BEV_HEIGHT],
                [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2], 
                [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2],
                [BEV_WIDTH - BEV_WIDTH/5, 0]
            ]).astype(np.int32)
        else:
            raise Exception("name should be front/back/left/right")
        return points
    
    # 마스크를 하나하나 채워주세요
    def get_mask(self, name):
        mask = np.zeros((BEV_HEIGHT,BEV_WIDTH), dtype=np.uint8)
        points = self.get_points(name)
        return cv2.fillPoly(mask, [points], 255)
    
    # 사전 설정 마스크의 겹치는 부분의 각 선분을 가져옵니다.
    def get_lines(self):
        self.lineFL = np.array([
                        [0, BEV_HEIGHT/5], 
                        [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2],
                    ]).astype(np.int32)
        self.lineFR = np.array([
                        [BEV_WIDTH, BEV_HEIGHT/5], 
                        [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2],
                    ]).astype(np.int32)
        self.lineBL = np.array([
                        [0, BEV_HEIGHT - BEV_HEIGHT/5], 
                        [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2],
                    ]).astype(np.int32)
        self.lineBR = np.array([
                        [BEV_WIDTH, BEV_HEIGHT - BEV_HEIGHT/5], 
                        [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2],
                    ]).astype(np.int32)
        self.lineLF = np.array([
                        [BEV_WIDTH/5, 0],
                        [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2]
                    ]).astype(np.int32)
        self.lineLB = np.array([
                        [BEV_WIDTH/5, BEV_HEIGHT],
                        [(BEV_WIDTH-CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2]
                    ]).astype(np.int32)
        self.lineRF = np.array([
                        [BEV_WIDTH - BEV_WIDTH/5, 0],
                        [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT-CAR_HEIGHT)/2]
                    ]).astype(np.int32)
        self.lineRB = np.array([
                        [BEV_WIDTH - BEV_WIDTH/5, BEV_HEIGHT],
                        [(BEV_WIDTH+CAR_WIDTH)/2, (BEV_HEIGHT+CAR_HEIGHT)/2]
                    ]).astype(np.int32)
        
    # 중첩점과 위의 선분 사이의 거리에 따라 그곳의 가중치 값이 구해집니다.
    def get_blend_mask(self, maskA, maskB, lineA, lineB):
        overlap = cv2.bitwise_and(maskA, maskB)           # 겹치는 영역
        indices = np.where(overlap != 0)                  # 중첩 영역의 좌표 지수
        for y, x in zip(*indices):
            distA = cv2.pointPolygonTest(np.array(lineA), (x, y), True)     # 겹치는 영역의 가장자리까지 거리 A
            distB = cv2.pointPolygonTest(np.array(lineB), (x, y), True)     # 겹치는 영역의 가장자리까지의 거리 B
            maskA[y, x] = distA**2 / (distA**2 + distB**2 + 1e-6) * 255     # 거리의 제곱비를 기준으로 가중치를 결정합니다.
        return maskA
    
    # 将图像乘以权重mask
    def __call__(self, img):
        return (img * self.weight).astype(np.uint8)    

In [ ]:
class BevGenerator:                   # 조감도 생성기 둘러보기
    def __init__(self, blend=args.BLEND_FLAG, balance=args.BALANCE_FLAG):
        self.init_args()
        self.cameras = [Camera('front'), Camera('back'), 
                        Camera('left'), Camera('right')]
        self.blend = blend
        self.balance = balance
        if not self.blend:
            self.masks = [Mask('front'), Mask('back'), 
                          Mask('left'), Mask('right')]
        else:
            self.masks = [BlendMask('front'), BlendMask('back'), 
                      BlendMask('left'), BlendMask('right')]
    
    # 매개변수를 수정하기 위해 외부 호출에 대한 args 매개변수를 가져옵니다.
    @staticmethod
    def get_args():
        return args
        
    # args 매개변수 재할당
    def init_args(self):
        global FRAME_WIDTH, FRAME_HEIGHT, BEV_WIDTH, BEV_HEIGHT
        global CAR_WIDTH, CAR_HEIGHT, FOCAL_SCALE, SIZE_SCALE
        FRAME_WIDTH = args.FRAME_WIDTH
        FRAME_HEIGHT = args.FRAME_HEIGHT
        BEV_WIDTH = args.BEV_WIDTH
        BEV_HEIGHT = args.BEV_HEIGHT
        CAR_WIDTH = args.CAR_WIDTH
        CAR_HEIGHT = args.CAR_HEIGHT
        FOCAL_SCALE = args.FOCAL_SCALE
        SIZE_SCALE = args.SIZE_SCALE
    
    # 전방, 후방, 좌측, 우측 카메라 원본 4개의 영상을 입력하면 조감도가 생성되며, 차량 영상은 입력할 필요가 없습니다.
    def __call__(self, front, back, left, right, car = None):
        images = [front,back,left,right]
        if self.balance:
            images = luminance_balance(images)        # 밝기 균형
        images = [mask(camera.raw2bev(img)) 
                  for img, mask, camera in zip(images, self.masks, self.cameras)]   # 조감도 변형 및 마스크 추가
        surround = cv2.add(images[0],images[1])
        surround = cv2.add(surround,images[2])
        surround = cv2.add(surround,images[3])        # 모든 이미지를 함께 결합
        if self.balance:
            surround = color_balance(surround)        # 화이트 밸런스
        if car is not None:
            surround = cv2.add(surround,car)          # 차량 사진 추가
        return surround

In [ ]:
def main():
    front = cv2.imread('./data/front/front.jpg')      # 전면 카메라 사진
    back = cv2.imread('./data/back/back.jpg')         # 후면 카메라 사진
    left = cv2.imread('./data/left/left.jpg')         # 왼쪽 카메라 사진
    right = cv2.imread('./data/right/right.jpg')      # 오른쪽 카메라 사진
    car = cv2.imread('./data/car.jpg')                # 차량 사진
    car = padding(car, BEV_WIDTH, BEV_HEIGHT)         # 조감도 크기에 맞게 차량 이미지에 검정색 테두리 추가
    
    bev = BevGenerator(blend=True,balance=True)       # 둘러보기 조감도 생성기 초기화
    surround = bev(front,back,left,right,car)         # 조감도를 얻으십시오
    
    cv2.namedWindow('surround', flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
    cv2.imshow('surround', surround)
    cv2.imwrite('./surround.jpg', surround)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == '__main__':
    main()